In [1]:
import sys
sys.path.append('../src')
from data_processing import load_data, clean_basic_data
from feature_engineering import create_all_features, extract_brands, create_rating_category

In [2]:
reviews, products = load_data()
cleaning_reviews = clean_basic_data(reviews)
merging_data = cleaning_reviews.merge(products, on='key', how='left')

Reviews with missing stars: 0
Started with 21674 reviews
After cleaning: 21674 reviews


In [3]:

merging_data['clean_text'] = merging_data['clean_text'].astype(str)

final_data = create_all_features(merging_data)
final_data = extract_brands(final_data)

final_data['sentiment_category'] = final_data['stars'].apply(create_rating_category)

Extracting text features...
clean_text sample: super good, don't get me wrong. but i came for the caramel and brownies, not the sweet cream. the packaging made it seem like brownies were packed and bountiful *crying frowny emoji* i'd say the taste of this was amazing, but the ratio of brownie to sweet cream was disappointing. liked it regardless but probably won't buy again simply because it didn't live up to its promising package. i'll find another one that has a better ratio and wayyy more yummy chewy brownies. overall, good flavor, texture, idea, and brownies. not so great caramel/sweet cream/ brownie ratio. just add more brownies. please.
Type: <class 'str'>


In [4]:
print(final_data['sentiment_category'].value_counts())
print(final_data['sentiment_category'].value_counts(normalize=True))

sentiment_category
positive    17079
negative     3535
neutral      1060
Name: count, dtype: int64
sentiment_category
positive    0.787995
negative    0.163099
neutral     0.048907
Name: proportion, dtype: float64


In [6]:
print(final_data[['clean_text', 'text_length', 'word_count']].head())#, 'spacy_polarity', 'spacy_subjectivity', 'vader_compound', 'vader_positive',  'vader_negative', 'vader_neutral']].head())

                                          clean_text  text_length  word_count
0  super good, don't get me wrong. but i came for...          603         102
1  i decided to try it out although i’m not a hug...          651         129
2  my caramel core begins to disappear about half...          715         140
3  why are people complaining about the blonde br...          565         106
4  this ice cream is worst ice cream i’ve ever ta...          623         120


In [7]:
#print(final_data[['clean_text', 'text_length', 'word_count', 'spacy_polarity', 'spacy_subjectivity', 'vader_compound', 'vader_positive',  'vader_negative', 'vader_neutral']].tail())

In [8]:
final_data = final_data.drop(columns=['title', 'author', 'text', 'subhead', 'description', 'taste', 'ingredients_x', 'ingredients_y','texture', 'likes', 'brand_y'])

In [9]:
final_data.to_csv('../data/processed/feature_dataset.csv', index=False)

print("Feature engineering done!!!")
print(f"Final dataset shape: {final_data.shape}")
print(f"Feature columns: {final_data.columns.tolist()}")

Feature engineering done!!!
Final dataset shape: (21674, 20)
Feature columns: ['brand_x', 'key', 'date', 'stars', 'helpful_yes', 'helpful_no', 'clean_text', 'name', 'rating', 'rating_count', 'brand_popularity', 'brand_average', 'text_length', 'word_count', 'sentence_count', 'exclamation_count', 'question_count', 'capital_ratio', 'avg_word_length', 'sentiment_category']


In [13]:
print("Feature Summary:")
print(final_data[['text_length', 'word_count']].describe())#, 'spacy_polarity', 'spacy_subjectivity']].describe())

print("\nMissing values:")
print(final_data.isnull().sum())

numeric_columns = ['rating', 'text_length', 'word_count']#, 'spacy_polarity', 'vader_compound', 'vader_positive']
correlction_matrix = final_data[numeric_columns].corr()
print("\nCorrelation with Ratings")
print(correlction_matrix['rating'].sort_values(ascending=False))

print("\nSentiment Methods Comparison:")
sample_reviews = final_data.head(5)
for idx, row in sample_reviews.iterrows():
    print(f"\nReview: {row['clean_text'][:100]}...")
    print(f"Ratings: {row['stars']}")
    #print(f"spaCy Polarity: {row['spacy_polarity']:.3f}")
    #print(f"VADER Compound: {row['vader_compound']:.3f}")


Feature Summary:
        text_length    word_count
count  21674.000000  21674.000000
mean     228.280059     42.772585
std      166.591031     31.041878
min       11.000000      1.000000
25%      117.000000     22.000000
50%      191.000000     36.000000
75%      281.000000     53.000000
max     2975.000000    514.000000

Missing values:
brand_x               0
key                   0
date                  0
stars                 0
helpful_yes           0
helpful_no            0
clean_text            0
name                  0
rating                0
rating_count          0
brand_popularity      0
brand_average         0
text_length           0
word_count            0
sentence_count        0
exclamation_count     0
question_count        0
capital_ratio         0
avg_word_length       0
sentiment_category    0
dtype: int64

Correlation with Ratings
rating         1.000000
word_count    -0.116388
text_length   -0.118414
Name: rating, dtype: float64

Sentiment Methods Comparison:

Review: 